In [4]:
%pip install beautifulsoup4

from bs4 import BeautifulSoup

with open("weather.html", "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

table_rows = soup.find("table").find("tbody").find_all("tr")
weather_data = []

for row in table_rows:
    columns = row.find_all("td")
    day = columns[0].text.strip()
    temp = int(columns[1].text.strip().replace("°C", ""))
    condition = columns[2].text.strip()
    weather_data.append({"day": day, "temperature": temp, "condition": condition})

print("Weather Forecast:")
for entry in weather_data:
    print(f"{entry['day']}: {entry['temperature']}°C, {entry['condition']}")

temp_max = max(weather_data, key=lambda x: x["temperature"])
print(f"\nDay with the highest temperature: {temp_max['day']} ({temp_max['temperature']}°C)")

sunny_days = [entry["day"] for entry in weather_data if entry["condition"] == "Sunny"]
print(f"Sunny days: {', '.join(sunny_days)}")

avg_temp = sum(entry["temperature"] for entry in weather_data) / len(weather_data)
print(f"\nAverage temperature for the week: {avg_temp:.2f}°C")

Note: you may need to restart the kernel to use updated packages.
Weather Forecast:
Monday: 25°C, Sunny
Tuesday: 22°C, Cloudy
Wednesday: 18°C, Rainy
Thursday: 20°C, Partly Cloudy
Friday: 30°C, Sunny

Day with the highest temperature: Friday (30°C)
Sunny days: Monday, Friday

Average temperature for the week: 23.00°C


In [6]:
import requests
import sqlite3
import pandas as pd
import json
from bs4 import BeautifulSoup


JOBS_URL = "https://realpython.github.io/fake-jobs"

DB_NAME = "jobs.db"

def create_database():
    """Create SQLite database and jobs table if not exists."""
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS jobs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            company TEXT,
            location TEXT,
            description TEXT,
            application_link TEXT,
            UNIQUE(title, company, location)
        )
    ''')
    conn.commit()
    conn.close()

def scrape_jobs():
    """Scrape job listings from the website."""
    response = requests.get(JOBS_URL)
    soup = BeautifulSoup(response.content, 'html.parser')
    jobs = []

    for job_element in soup.find_all('div', class_='card-content'):
        title_element = job_element.find('h2', class_='title')
        company_element = job_element.find('h3', class_='company')
        location_element = job_element.find('p', class_='location')
        description_element = job_element.find('div', class_='description')
        application_link_element = job_element.find('a', text='Apply')

        title = title_element.text.strip() if title_element else "N/A"
        company = company_element.text.strip() if company_element else "N/A"
        location = location_element.text.strip() if location_element else "N/A"
        description = description_element.text.strip() if description_element else "N/A"
        application_link = application_link_element['href'] if application_link_element else "N/A"

        jobs.append({
            "title": title,
            "company": company,
            "location": location,
            "description": description,
            "application_link": application_link
        })
    
    return jobs

def save_jobs_to_db(jobs):
    """Insert scraped jobs into SQLite database with incremental updates."""
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    for job in jobs:
        cursor.execute('''
            INSERT INTO jobs (title, company, location, description, application_link)
            VALUES (?, ?, ?, ?, ?)
            ON CONFLICT(title, company, location) DO UPDATE SET
                description=excluded.description,
                application_link=excluded.application_link
        ''', (job['title'], job['company'], job['location'], job['description'], job['application_link']))

    conn.commit()
    conn.close()

def filter_jobs(location=None, company=None):
    """Retrieve jobs based on filters."""
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    query = "SELECT * FROM jobs WHERE 1=1"
    params = []

    if location:
        query += " AND location = ?"
        params.append(location)
    if company:
        query += " AND company = ?"
        params.append(company)

    cursor.execute(query, params)
    results = cursor.fetchall()
    conn.close()
    return results

def export_jobs_to_csv(jobs, filename="filtered_jobs.csv"):
    """Export filtered jobs to CSV."""
    df = pd.DataFrame(jobs, columns=['id', 'title', 'company', 'location', 'description', 'application_link'])
    df.to_csv(filename, index=False)

create_database()
jobs = scrape_jobs()
save_jobs_to_db(jobs)


filtered_jobs = filter_jobs(location="Remote")
export_jobs_to_csv(filtered_jobs)
print("Job data has been scraped and stored successfully.")


Job data has been scraped and stored successfully.


C:\Users\posit\AppData\Local\Temp\ipykernel_18188\3972501434.py:41: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  application_link_element = job_element.find('a', text='Apply')


In [7]:
import requests
import json
from bs4 import BeautifulSoup


DEMOBLAZE_URL = "https://www.demoblaze.com/"

def scrape_laptops():
    """Scrape laptop data from Demoblaze."""
    response = requests.get(DEMOBLAZE_URL)
    soup = BeautifulSoup(response.content, 'html.parser')

    laptops = []
    laptop_elements = soup.find_all('div', class_='card')

    for laptop_element in laptop_elements:
        name = laptop_element.find('h4', class_='card-title').a.text.strip()
        price = laptop_element.find('h5').text.strip()
        description = laptop_element.find('p', class_='card-text').text.strip()

        laptops.append({
            "name": name,
            "price": price,
            "description": description
        })

    return laptops

def save_laptops_to_json(laptops, filename="laptops.json"):
    """Save scraped laptop data to JSON file."""
    with open(filename, "w") as json_file:
        json.dump(laptops, json_file, indent=4)


laptops = scrape_laptops()
save_laptops_to_json(laptops)

print("Laptop data has been scraped and stored in laptops.json successfully.")

Laptop data has been scraped and stored in laptops.json successfully.
